In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow import math
import numpy

numpy.random.seed(10)

In [11]:
vocab_size = 500
model_dimension = 128

# Define the LSTM modle
LSTM = Sequential()
LSTM.add(layers.Embedding(input_dim = vocab_size, output_dim = model_dimension))
LSTM.add(layers.LSTM(units = model_dimension, return_sequences = True))
LSTM.add(layers.AveragePooling1D(pool_size = 2))
LSTM.add(layers.Lambda(lambda x : math.l2_normalize(x)))

input_1 = layers.Input((None,))
input_2 = layers.Input((None,))

# Concatenate two LSTMs together
conc = layers.Concatenate(axis = 1)((LSTM(input_1), LSTM(input_2)))

# Use the parallel combinator to create a Siamese model out of the LSTM
Siamese = Model(inputs = (input_1, input_2), outputs = conc)

# Print the summary of the model
Siamese.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_7             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_6 (Sequential) │ (None, None, 128)      │        195,584 │ input_layer_6[0][0],   │
│                           │                        │                │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, None, 128)      │              0 │ sequential_6[0][0],    │
│ (Concatenate)             │                        │                │ sequential_6[1][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 195,584 (764.00 KB)

 Trainable params: 195,584 (764.00 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
def show_layers(model, layer_prefix):
  print("Total layers : ", len(model.layers), "\n")
  for i in range(len(model.layers)):
    print("========")
    print(f"{layer_prefix}_{i} : {model.layers[i]}\n")

print("Siamese model : \n")
show_layers(Siamese, 'Parallel.sublayers')

print("Details of LSTM models: \n")
show_layers(LSTM, "Serial.sublayers")

Siamese model : 

Total layers :  4 

Parallel.sublayers_0 : <InputLayer name=input_layer_6, built=True>

Parallel.sublayers_1 : <InputLayer name=input_layer_7, built=True>

Parallel.sublayers_2 : <Sequential name=sequential_6, built=True>

Parallel.sublayers_3 : <Concatenate name=concatenate_2, built=True>

Details of LSTM models: 

Total layers :  4 

Serial.sublayers_0 : <Embedding name=embedding_6, built=True>

Serial.sublayers_1 : <LSTM name=lstm_6, built=True>

Serial.sublayers_2 : <AveragePooling1D name=average_pooling1d_3, built=True>

Serial.sublayers_3 : <Lambda name=lambda_2, built=True>

